In [5]:
!pip install chromadb

In [6]:
import pandas as pd

In [7]:
url = "https://raw.githubusercontent.com/jogfx/MLops-exam/main/job_listings_cleaned.csv"
df = pd.read_csv(url)

In [8]:
df.head()

,Entry_id,Title,Description,Published Date,Location,URL
0,h1459680,Kollega til forligskontoret i Forsvarsminister...,Vil du være med til at sætte retningen for for...,2024-04-11,København K,https://www.jobindex.dk/vis-job/h1459680
1,h1459679,Elteknisk supporter til Campus Service i Taastrup,NaN,2024-04-11,Taastrup,https://www.jobindex.dk/vis-job/h1459679
2,h1459675,Serviceassistent,Vi søger hjælp til at holde pænt og rent i vor...,2024-04-11,Kastrup,https://www.jobindex.dk/vis-job/h1459675
3,h1459673,Chef for Økonomistyring og Regnskab,Vil du både lede økonomistyring på et strategi...,2024-04-11,København Ø,https://www.jobindex.dk/vis-job/h1459673
4,h1459671,Maskinmester til spildevandsafdeling generati...,Vil du være med til at drive og optimere afløb...,2024-04-11,København S,https://www.jobindex.dk/vis-job/h1459671


In [9]:
# Importing necessary libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
import nltk

In [10]:
# Downloading necessary files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
# Function to preprocess text
def tokenize_and_lowercase(text):
    return [token.lower() for token in word_tokenize(text)]

def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def preprocess_text(text):
    tokens = tokenize_and_lowercase(text)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize_words(tokens)
    return ' '.join(tokens)

In [12]:
# Preprocessing text columns
text_columns = ['Title', 'Description']

for column in text_columns:
    df[column] = df[column].astype(str).apply(preprocess_text)

In [13]:
df.head()

,Entry_id,Title,Description,Published Date,Location,URL
0,h1459680,kollega til forligskontoret forsvarsministerie...,vil du være med til sætte retningen forsvarsfo...,2024-04-11,København K,https://www.jobindex.dk/vis-job/h1459680
1,h1459679,elteknisk supporter til campus service taastrup,nan,2024-04-11,Taastrup,https://www.jobindex.dk/vis-job/h1459679
2,h1459675,serviceassistent,vi søger hjælp til holde pænt og rent vort hus...,2024-04-11,Kastrup,https://www.jobindex.dk/vis-job/h1459675
3,h1459673,chef økonomistyring og regnskab,vil du både lede økonomistyring på et strategi...,2024-04-11,København Ø,https://www.jobindex.dk/vis-job/h1459673
4,h1459671,maskinmester til spildevandsafdeling generatio...,vil du være med til drive og optimere afløbssy...,2024-04-11,København S,https://www.jobindex.dk/vis-job/h1459671


In [14]:
import csv

In [15]:
# Writing the preprocessed data to a CSV file
with open('job_listings_cleaned.csv') as f:
    lines = csv.reader(f)

    documents = []
    metadatas = []
    ids = []
    id = 1

    for i, line in enumerate(lines):
        if i == 0:
            continue
    
        documents.append(line[1])
        metadatas.append({'Entry_id': line[0]})
        ids.append(str(id))
        id += 1

In [16]:
# Importing the Chroma SDK
import chromadb
chroma_client = chromadb.Client()

In [17]:
# Creating a collection
collection = chroma_client.create_collection('job_listings')

In [18]:
# Adding documents to the collection
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [21]:
results = collection.query(query_texts=['data scientist'], n_results=5)

In [22]:
results

{'ids': [['5541', '2482', '11440', '11774', '9229']],
 'distances': [[0.0,
   0.30933678150177,
   0.3827909529209137,
   0.3827909529209137,
   0.38855043053627014]],
 'metadatas': [[{'Entry_id': 'h1454261'},
   {'Entry_id': 'h1457323'},
   {'Entry_id': 'r12404492'},
   {'Entry_id': 'r12404603'},
   {'Entry_id': 'r12414784'}]],
 'embeddings': None,
 'documents': [['Data Scientist',
   'Data science student',
   'Senior Data Scientist',
   'Senior Data Scientist',
   'Data Analyst']],
 'uris': None,
 'data': None}